# Background

### - As companies grow over time, so does the amount of data they own. Because of this, it becomes increasingly necessary to develop efficient algorithms and update existing ones. One such example is using a binary search over a linear search. A linear search involves looking up elements in a list and returning the first match whereas a binary search keeps dividing a (sorted) list in half until its exhausted.

### - Consider this. Do you remember when you were younger we had those big phone books? If you wanted to look up the number for Nike, you wouldn't start on the first page, right? You would likely open the book somewhere close to the middle, and continue dividing the rest in half until you found it. This is a binary search. It is a smarter approach for looking up data.

# Problem

### - What is the Big O notation for searching an element in an array list? How does this compare to a binary search?

##### - _Answer: O(n); 0(log N)_

# Step 1: Use Spark to read and structure our data

In [1]:
from pyspark.sql import SparkSession, dataframe
from pandas.core import frame, series
from typing import List
from time import time 

In [2]:
appName = 'nike'
pto = 'shoes.csv'
user_input_example_true = 'Nike Pegasus Turbo'
user_input_example_false = 'Nike Pegisus Turbo'

In [3]:
def build_spark_session() -> SparkSession:
    return SparkSession.builder.appName(appName).getOrCreate()

def read_shoe_data() -> dataframe.DataFrame:
    return spark.read.csv(pto, sep=',', inferSchema=True, header=True)

def convert_to_pandas_df(shoes: dataframe.DataFrame) -> frame.DataFrame:
    return shoes.toPandas()

def extract_shoes_from_df(df: frame.DataFrame) -> series.Series:
    return df.shoes

def convert_series_to_list(shoes_series: series.Series) -> List[str]:
    return list(shoes_series)

In [4]:
spark = build_spark_session()
shoes = read_shoe_data()
df = convert_to_pandas_df(shoes)
shoes_series = extract_shoes_from_df(df)
shoes_list = convert_series_to_list(shoes_series)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/02 18:04:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/02 18:04:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , shoes
 Schema: _c0, shoes
Expected: _c0 but found: 
CSV file: file:///Users/aaronestes/projects/nike/shoes.csv


# Step 2: Compare Performance of Linear Search with Binary Search

In [5]:
def timer(func): 
    def wrap_func(*args, **kwargs): 
        t1 = time() 
        result = func(*args, **kwargs) 
        t2 = time() 
        print(f'Function {func.__name__!r} executed in {float(t2-t1)} seconds') 
        return result 
    return wrap_func 

In [6]:
## linear search
@timer
def one_at_a_time(shoes, shoe):
    for s in shoes:
        if s == shoe:
            return True
    return False

In [7]:
## binary search
@timer
def divide_and_conquer(shoes, shoe):
    left = 0
    right = len(shoes) - 1

    while left <= right:
        middle = (left + right) // 2

        if shoe > shoes[middle]:
            left = middle + 1
        elif shoe < shoes[middle]:
            right = middle - 1
        else:
            return True
    return False

In [8]:
## compare the perfomance when the input is valid
is_shoe_valid_linear_search_match = one_at_a_time(shoes_list, user_input_example_true)
is_shoe_valid_binary_search_match = divide_and_conquer(shoes_list, user_input_example_true)

Function 'one_at_a_time' executed in 2.86102294921875e-06 seconds
Function 'divide_and_conquer' executed in 2.1457672119140625e-06 seconds


In [9]:
## compare the perfomance when the input is invalid
is_shoe_valid_linear_search_no_match = one_at_a_time(shoes_list, user_input_example_false)
is_shoe_valid_binary_search_no_match = divide_and_conquer(shoes_list, user_input_example_false)

Function 'one_at_a_time' executed in 4.0531158447265625e-06 seconds
Function 'divide_and_conquer' executed in 2.1457672119140625e-06 seconds


In [10]:
def stop_spark_session(spark: SparkSession):
    spark.stop()

stop_spark_session(spark)

# Conclusion

### - We can see that the binary search algorithm takes less time in both cases... when the input has a match AND when the input does not match. Although the dataset we are using is small, less than 300 elements, the effect becomes much more noticeable when dealing with datasets in the millions. Given that the list is sorted beforehand, it would be too expensive for us not to use the binary search.